# MRE: Federal CTC Expansion Reduces MA State Tax

This notebook demonstrates how federal CTC expansion can reduce Massachusetts state tax by triggering itemization.

In [ ]:
from policyengine_us import Simulation
from policyengine_us.model_api import *

## Define a minimal tax unit

In [ ]:
# Create a simple family in MA with medical expenses
situation = {
    "people": {
        "parent": {
            "age": {2026: 40},
            "employment_income": {2026: 50_000},
            "medical_out_of_pocket_expenses": {2026: 4_000},
        },
        "child": {
            "age": {2026: 10},
        },
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent", "child"],
            "tax_unit_childcare_expenses": {2026: 0},
        }
    },
    "households": {
        "household": {
            "members": ["parent", "child"],
            "state_code": {2026: "MA"},
        }
    },
}

## Scenario 1: Without CTC Expansion

In [ ]:
# Reform stack WITHOUT CTC
reform_no_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.1": {"2026-01-01": 0.12},
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.25},
    "gov.irs.deductions.standard.amount.SINGLE": {"2026-01-01": 16_550},
}, country_id="us")

sim_no_ctc = Simulation(situation=situation, reform=reform_no_ctc)

print("WITHOUT CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_no_ctc.calculate('tax_unit_itemizes', 2026)[0])}")
print(f"  Federal tax (itemizing): ${sim_no_ctc.calculate('tax_liability_if_itemizing', 2026)[0]:,.2f}")
print(f"  Federal tax (standard): ${sim_no_ctc.calculate('tax_liability_if_not_itemizing', 2026)[0]:,.2f}")
print(f"  MA state tax: ${sim_no_ctc.calculate('ma_income_tax', 2026)[0]:,.2f}")
print(f"  MA Part B exemption: ${sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]:,.2f}")

## Scenario 2: With CTC Expansion

In [ ]:
# Add CTC expansion
reform_with_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.1": {"2026-01-01": 0.12},
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.25},
    "gov.irs.deductions.standard.amount.SINGLE": {"2026-01-01": 16_550},
    # CTC expansion
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 3_000},
    "gov.irs.credits.ctc.refundable.individual_max": {"2026-01-01": 2_000},
}, country_id="us")

sim_with_ctc = Simulation(situation=situation, reform=reform_with_ctc)

print("WITH CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_with_ctc.calculate('tax_unit_itemizes', 2026)[0])}")
print(f"  Federal tax (itemizing): ${sim_with_ctc.calculate('tax_liability_if_itemizing', 2026)[0]:,.2f}")
print(f"  Federal tax (standard): ${sim_with_ctc.calculate('tax_liability_if_not_itemizing', 2026)[0]:,.2f}")
print(f"  MA state tax: ${sim_with_ctc.calculate('ma_income_tax', 2026)[0]:,.2f}")
print(f"  MA Part B exemption: ${sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]:,.2f}")

## Analysis

In [ ]:
# Calculate the impact
ma_tax_no_ctc = sim_no_ctc.calculate('ma_income_tax', 2026)[0]
ma_tax_with_ctc = sim_with_ctc.calculate('ma_income_tax', 2026)[0]
ma_tax_change = ma_tax_with_ctc - ma_tax_no_ctc

exemption_no_ctc = sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]
exemption_with_ctc = sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]
exemption_change = exemption_with_ctc - exemption_no_ctc

print("\nIMPACT OF FEDERAL CTC EXPANSION:")
print(f"  MA state tax change: ${ma_tax_change:,.2f}")
print(f"  MA Part B exemption change: ${exemption_change:,.2f}")

if exemption_change > 0:
    print(f"\nMECHANISM:")
    print(f"1. CTC expansion may trigger federal itemization")
    print(f"2. MA allows itemizers to claim medical expense deduction")
    print(f"3. Medical expense deduction: ${sim_with_ctc.calculate('medical_expense_deduction', 2026)[0]:,.2f}")
    print(f"4. MA tax rate: 5%")
    print(f"5. Expected tax savings: ${exemption_change:,.2f} × 5% = ${exemption_change * 0.05:,.2f}")